URL
PowerPoint
ReadTheDocs site
PDF

In [4]:
import os
from langchain.document_loaders import UnstructuredURLLoader, UnstructuredPowerPointLoader, ReadTheDocsLoader, \
    PyPDFLoader
from langchain.llms import OpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.callbacks import get_openai_callback
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
from dotenv import load_dotenv
import os
from langchain_community.llms.cloudflare_workersai import CloudflareWorkersAI
from langchain_community.llms.tongyi import Tongyi
from langchain_openai import ChatOpenAI

load_dotenv(override=True)

account_id = os.getenv('CF_ACCOUNT_ID')
api_token = os.getenv('CF_API_TOKEN')
print(account_id)
print(api_token)

# CloudflareWorkersAI
model = '@cf/meta/llama-3-8b-instruct'
cf_llm = CloudflareWorkersAI(
    account_id=account_id,
    api_token=api_token,
    model=model
)

DASHSCOPE_API_KEY = os.getenv('DASHSCOPE_API_KEY')
print(DASHSCOPE_API_KEY)

# qwen
qw_llm = Tongyi(
    model='qwen2-1.5b-instruct'
)

# qwen 兼容 openai的接口
qw_llm_openai = ChatOpenAI(
    openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1',
    openai_api_key=DASHSCOPE_API_KEY,
    model_name="qwen2-1.5b-instruct",
    temperature=0,
    streaming=True,
    verbose=True,
)

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv('OPENAI_API_BASE')
print(api_key)
print(base_url)

# openai/moonshot
ms_llm = ChatOpenAI(
    openai_api_base=base_url,
    openai_api_key=api_key,
    model_name="moonshot-v1-8k",
    temperature=0.7,
)

8483c3ec7a0cbc54a8d660b5b9002b04
Gcllof8ze6dgtcqFI5FQZ2SD_5tfCD4Db7NuS6jn
sk-01c5003340c3453b934052d737d45e01
sk-UGVpjuTwo2Q8pewoqUDfckw1A0pbSDli9ElFMeS9WareKknG
https://api.moonshot.cn/v1/


In [7]:
def summarize_docs(docs, doc_url):
    print(f'You have {len(docs)} document(s) in your {doc_url} data')
    print(f'There are {len(docs[0].page_content)} characters in your document')

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
    split_docs = text_splitter.split_documents(docs)

    print(f'You have {len(split_docs)} split document(s)')

    chain = load_summarize_chain(qw_llm_openai, chain_type="map_reduce", verbose=False)

    response = ""
    with get_openai_callback() as cb:
        response = chain.invoke(split_docs)
        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Successful Requests: {cb.successful_requests}")
        print(f"Total Cost (USD): ${cb.total_cost}")

    return response

In [6]:
url = "https://www.crab233.cloudns.biz/proxy/edition.cnn.com/2023/04/13/business/delta-earnings/index.html"
loader = UnstructuredURLLoader(urls=[url]).load()

In [8]:
summarize_docs(loader, url)

You have 1 document(s) in your https://www.crab233.cloudns.biz/proxy/edition.cnn.com/2023/04/13/business/delta-earnings/index.html data
There are 5159 characters in your document
You have 6 split document(s)
Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


{'input_documents': [Document(page_content="Video Ad Feedback\n\nAirlines warn they may have to cut flights unless this problem is solved\n\n02:26\n\nSource:\n                CNN\n\nTop business news\n\n16 videos\n\nVideo Ad Feedback\n\nAirlines warn they may have to cut flights unless this problem is solved\n\n02:26\n\nNow playing\n\nSource:\n                CNN\n\nVideo Ad Feedback\n\nHow to craft the perfect maternity leave out of office message\n\n01:33\n\nNow playing\n\nSource:\n                CNN\n\nVideo Ad Feedback\n\nCNN writer explains how Microsoft's new AI model works\n\n02:20\n\nNow playing\n\nSource:\n                CNN\n\nVideo Ad Feedback\n\nToyota sells only two electric vehicles. Executive insists they're not holding back\n\n02:21\n\nNow playing\n\nSource:\n                CNN\n\nVideo Ad Feedback\n\nAn implant in his brain lets him do incredible tasks with his thoughts\n\n05:37\n\nNow playing\n\nSource:\n                CNN\n\nVideo Ad Feedback\n\n'We're cooking ou

In [5]:
url2 = 'https://mp.weixin.qq.com/s/mnG40InZ1vBSkXjV50B3mQ'
loader = UnstructuredURLLoader(urls=[url2]).load()

In [8]:
summarize_docs(loader, url2)

You have 1 document(s) in your https://mp.weixin.qq.com/s/mnG40InZ1vBSkXjV50B3mQ data
There are 64 characters in your document
You have 1 split document(s)
Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


{'input_documents': [Document(page_content='环境异常\n\n当前环境异常，完成验证后即可继续访问。\n\n去验证\n\n视频\n\n小程序\n\n，轻点两下取消赞\n\n在看\n\n，轻点两下取消在看', metadata={'source': 'https://mp.weixin.qq.com/s/mnG40InZ1vBSkXjV50B3mQ'})],
 'output_text': '"请先进行验证以继续访问。点击\'去验证\'页面并取消点赞或在看两次。"'}

In [1]:
!wget "https://github.com/tomw1808/truffle_eth_class2/blob/master/s08/Web3-intro.pptx?raw=true" -O Web3-intro.pptx

--2024-07-02 09:30:24--  https://github.com/tomw1808/truffle_eth_class2/blob/master/s08/Web3-intro.pptx?raw=true
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://github.com/tomw1808/truffle_eth_class2/raw/master/s08/Web3-intro.pptx [following]
--2024-07-02 09:30:26--  https://github.com/tomw1808/truffle_eth_class2/raw/master/s08/Web3-intro.pptx
Reusing existing connection to github.com:443.
HTTP request sent, awaiting response... No data received.
Retrying.

--2024-07-02 09:30:57--  (try: 2)  https://github.com/tomw1808/truffle_eth_class2/raw/master/s08/Web3-intro.pptx
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/tomw1808/truffle_eth_class2/master/s08/Web3-intro.pptx [following]
--2024-07-02 09:30:58--  https://raw.githubusercontent

In [ ]:
loader = UnstructuredPowerPointLoader("Web3-intro.pptx")
response = summarize_docs(loader.load(), "Web3-intro.pptx")
print(response)

In [ ]:
!wget -r -A.html -P langchain "https://langchain.readthedocs.io/en/latest/"

In [ ]:
# 获取文件夹内的文件
loader = ReadTheDocsLoader("langchain")
summarize_docs(loader.load(), "langchain")

In [ ]:
!wget "https://ir.tesla.com/_flysystem/s3/sec/000095017023001409/tsla-20221231-gen.pdf" -O tsla-20221231-gen.pdf

In [ ]:
loader = PyPDFLoader("tsla-20221231-gen.pdf")
pages = loader.load_and_split()
summarize_docs(pages[:10], "tsla-20221231-gen.pdf")

In [16]:
# 获取文件夹内的文件
loader = ReadTheDocsLoader("langchain")
loader = loader.load()

In [17]:
summarize_docs(loader, "03_data_loader")

You have 0 document(s) in your 03_data_loader data


IndexError: list index out of range

In [18]:
from langchain.document_loaders import TextLoader

loader = TextLoader("../03_data_loader/test.md")
docs = loader.load()

In [19]:
summarize_docs(docs, "03_data_loader")

You have 1 document(s) in your 03_data_loader data
There are 5367 characters in your document
You have 6 split document(s)
Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Successful Requests: 0
Total Cost (USD): $0.0


{'input_documents': [Document(page_content='# 呼伦贝尔秋摄丨从大兴安岭到中俄边境8日\n\n入住阿尔山景区丨驼峰岭天池丨巴彦呼硕丨根河湿地丨白鹿岛丨莫尔道嘎丨莫日格勒河丨秘境呼伦湖丨中俄边境公路丨安排日出日落\n跟团游 · 摄影采风 · 小众风光 · 季节限定\n\n## 早鸟特惠\n\n活动时间：2024-06-21 00:00:00 - 2024-08-10 00:00:00\n*以付款时间到出行时间间隔为准\n提前45天下单并支付，立减50元/人\n提前60天下单并支付，立减100元/人\n产品价格： ￥ 4980 元起（成人）\n\n## 满减优惠\n\n活动时间：2023-07-19 14:00:00 - 2025-08-30 23:59:59\n满6人, 优惠50元/人\n满10人, 优惠100元/人\n满20人, 优惠150元/人\n\n产品特色： 专业摄影领队 充足拍摄时间 摄影题材丰富 秋季限时发团\n供 应 商： 游侠客国际旅游有限公司\n出 发 地： 海拉尔\n退订政策： 支持退款 特殊原因退订保障 查看政策 >\n出团通知： 周边游提前1天，国内游提前3天，出境游提前3-5天，APP和短信群发出团通知\n行程套餐： 呼伦贝尔秋摄8日\n批次套餐： 2024海拉尔集散\n\n## 费用包含\n\n### 成人\n\n交通：D2-D8包车费、油费、停车费、路桥费、司机食宿补贴（每车上限22人左右，空座率＞30%），海拉尔-巴彦呼硕-草原穿越小车包车费（1人1座）\n住宿：行程中所列7晚酒店标间\n餐食：全程酒店含早餐\n门票及景交：根河源湿地门票、莫尔道嘎国家森林公园门票+小火车、白鹿岛门票、白桦林门票+景交、186彩带河门票+景交、阿尔山大门票+环保车、满洲里国门景区门票\n体验项目：黑山头马场，安排套马等拍摄活动费用\n随团服务人员：全程导游、摄影领队带队费及食宿补贴\n优惠补充说明：如持有军官证、伤残证、学生证、教师证等特殊证件，请在下单时备注，并携带原件，在团上向服务人员出示。因旅行社在景区门票上面是折扣票，优惠减免是按旅行社折扣价退还，不以景区挂牌价退还，优惠金额以景区政策为准。活动结束后线上退款/现场退款\n\n### 自理费用', metadata={'source': '../03_d

In [29]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel


# Add typing for input
class Question(BaseModel):
    __root__: str


prompt = """ 这是 prompt {__root__}"""
# 输入实体化
chain = ChatPromptTemplate.from_template(prompt) | qw_llm_openai
chain = chain.with_types(input_type=Question)

In [30]:
chain.invoke(Question(__root__="What is the capital of France?"))

AIMessage(content="It appears you have entered a Python code snippet and I'm unable to understand it due to the lack of context. Could you please provide more information or clarify what you need help with?", response_metadata={'finish_reason': 'stop'}, id='run-77824df2-b0f0-41c8-a6e5-01f17fade4ea-0')